In [ ]:
import wikipediaapi
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from tkinter import Tk, Label, Button

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Create a Wikipedia API object
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI,
    user_agent="YourAppName/1.0"
)

def fetch_wikipedia_summary(query):
    """
    Fetches the Wikipedia summary for a given query.

    Args:
        query (str): The search query.

    Returns:
        str: The Wikipedia summary text if found, or an error message.
    """
    page = wiki_wiki.page(query)
    if page.exists():
        return page.text
    else:
        return "Sorry, I couldn't find any information on that topic."

def preprocess_query(query):
    """
    Preprocesses the user's query by tokenizing and removing stopwords.

    Args:
        query (str): The user's query.

    Returns:
        str: The processed query text.
    """
    tokens = word_tokenize(query)
    
    stopwords_list = set(stopwords.words('english'))
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stopwords_list]
    
    processed_query = TreebankWordDetokenizer().detokenize(tokens)
    
    return processed_query

def generate_pdf(text, filename):
    """
    Generates a PDF document with properly formatted content.

    Args:
        text (str): The content to be included in the PDF.
        filename (str): The name of the PDF file to be generated.
    """
    doc = SimpleDocTemplate(filename, pagesize=letter)
    story = []

    text_lines = text.split('\n')
    styles = getSampleStyleSheet()
    normal_style = styles['Normal']

    for line in text_lines:
        p = Paragraph(line, normal_style)
        story.append(p)
        story.append(Spacer(1, 12))  # Add some space between paragraphs

    doc.build(story)

def thank_you_window():
    """
    Display a thank you message in a new GUI window.
    """
    window = Tk()
    window.title("Thank You")
    label = Label(window, text="Thank you for using BINFROW by Siddhant")
    label.pack()
    window.mainloop()

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break
    
    processed_query = preprocess_query(user_input)
    if processed_query:
        result = fetch_wikipedia_summary(processed_query)
        print("Chatbot:", result)
        
        pdf_input = input("Do you want a PDF for this? (y/n): ")
        if pdf_input.lower() == 'y':
            pdf_filename = f"{processed_query.replace(' ', '_')}_info.pdf"
            generate_pdf(result, pdf_filename)
            print(f"Information saved as {pdf_filename}")
        
        continue_input = input("Do you want to continue? (y/n): ")
        if continue_input.lower() != 'y':
            thank_you_window()
            break
    else:
        print("Chatbot: Please provide a valid query.")
